In [ ]:
# 마이크로소프트 오피스 PPT, Excel, Word 읽기
# pip install unstructured openpyxl python-magic python-pptx
# pip install "unstructured[all-docs]"
# pip install -U nltk

# 파일로드 관련 설명서
# https://python.langchain.com/docs/integrations/providers/unstructured/



In [1]:
import nltk
import shutil
import os

# 실행 후에 C:\Users\Administrator\AppData\Roaming\nltk_data\tokenizers\punkt\PY3_tab 이 없다면
# PY3 를 PY3_tab으로 복사해서 사용

# 해당 폴더가 없을때만 설치코드 실행
nltk_dir = 'C:\\Users\\Administrator\\AppData\\Roaming\\nltk_data\\tokenizers\\punkt'
if not os.path.exists(nltk_dir):
    print('nltk_data 다운로드 시작')
    nltk.download('punkt')

source_folder = nltk_dir + '\\PY3'
target_folder = nltk_dir + '\\PY3_tab'

# Check if the target folder exists
if os.path.exists(source_folder):
    if not os.path.exists(target_folder):
        # Create the target folder
        os.makedirs(target_folder)
        print(f"PY3_tab 폴더를 새로 생성했습니다: {target_folder}")

        # Copy the files from the source folder to the target folder
        for file_name in os.listdir(source_folder):
            source_file = os.path.join(source_folder, file_name)
            target_file = os.path.join(target_folder, file_name)
            shutil.copy2(source_file, target_file)
            print(f"파일복사 : {file_name}")
    else:
        print(f"PY3_tab 폴더가 이미 존재합니다: {target_folder}")
else:
    print(f"nltk 폴더가 없습니다. 다운로드 해주세요: {source_folder}")


PY3_tab 폴더가 이미 존재합니다: C:\Users\Administrator\AppData\Roaming\nltk_data\tokenizers\punkt\PY3_tab


In [2]:
# PPT 읽고 대본 생성시키기
from langchain_community.document_loaders import UnstructuredPowerPointLoader

loader = UnstructuredPowerPointLoader("data/ml_course.pptx", mode="elements")

docs = loader.load()

len(docs)

48

In [3]:
docs

[Document(metadata={'source': 'data/ml_course.pptx', 'category_depth': 0, 'file_directory': 'data', 'filename': 'ml_course.pptx', 'last_modified': '2025-02-14T07:31:29', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'category': 'Title', 'element_id': '1378fc8ea4dd0830a3c5f93e9d31a516'}, page_content='Machine Learning Model Deployment'),
 Document(metadata={'source': 'data/ml_course.pptx', 'category_depth': 1, 'file_directory': 'data', 'filename': 'ml_course.pptx', 'last_modified': '2025-02-14T07:31:29', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.presentationml.presentation', 'parent_id': '1378fc8ea4dd0830a3c5f93e9d31a516', 'category': 'Title', 'element_id': '5de650940883c293416c7f769aa0c961'}, page_content='Introduction to ML Pipeline'),
 Document(metadata={'source': 'data/ml_course.pptx', 'category_depth': 1, 'file_directory': 'data', 'filename': '

In [4]:
# 같은 페이지 내용은 딕셔너리로 묶고 내용마다 \n\n 으로 구분했음
ppt_data = {}
for doc in docs:
    page = doc.metadata["page_number"]
    ppt_data[page] = ppt_data.get(page, "") + "\n\n" + doc.page_content

In [5]:
ppt_data

{1: '\n\nMachine Learning Model Deployment\n\nIntroduction to ML Pipeline\n\nhttps://bit.ly/bert_nlp\n\n한국어\n\n',
 2: '\n\nWhat is Machine Learning Pipeline?\n\n',
 3: '\n\nType of ML Deployment\n\nBatch: In batch deployment, ML models process large volumes of data at scheduled intervals, ideal for tasks like end-of-day reporting or monthly analytics.\n\nStream: Stream deployment enables ML models to process and analyze data in real-time as it flows in, suitable for applications like fraud detection or live social media analysis.\n\nRealtime: Realtime deployment allows ML models to provide instant predictions or decisions in response to incoming data, essential for use cases like recommendation systems or autonomous driving.\n\nEdge: Edge deployment involves running ML models on local devices close to the data source, reducing latency and bandwidth usage, which is crucial for IoT applications and smart devices.\n\n',
 4: '\n\nInfrastructure and Integration\n\nHardware and Software: Set

In [6]:
# 딕셔너리에서 텍스트로 가져온다
# strip()으로 앞뒤 공백 제거하고 수동으로 \n\n추가

context = ""
for page, content in ppt_data.items():
    context += f"### Slide {page}:\n\n{content.strip()}\n\n\n"

In [7]:
print(context)

### Slide 1:

Machine Learning Model Deployment

Introduction to ML Pipeline

https://bit.ly/bert_nlp

한국어


### Slide 2:

What is Machine Learning Pipeline?


### Slide 3:

Type of ML Deployment

Batch: In batch deployment, ML models process large volumes of data at scheduled intervals, ideal for tasks like end-of-day reporting or monthly analytics.

Stream: Stream deployment enables ML models to process and analyze data in real-time as it flows in, suitable for applications like fraud detection or live social media analysis.

Realtime: Realtime deployment allows ML models to provide instant predictions or decisions in response to incoming data, essential for use cases like recommendation systems or autonomous driving.

Edge: Edge deployment involves running ML models on local devices close to the data source, reducing latency and bandwidth usage, which is crucial for IoT applications and smart devices.


### Slide 4:

Infrastructure and Integration

Hardware and Software: Setting up 

In [8]:
### LLM 모듈 가져오기
from my_scripts import my_llm

In [9]:
# PPT 요약 스크립트(대본) 작성 요청

# 위에 제공된 각 PowerPoint 슬라이드에 대해 핵심 요점을 효과적으로 전달하는 2분 분량의 스크립트를 작성하세요.
# 슬라이드 간의 원활한 흐름을 보장하고 명확하고 매력적인 내러티브를 유지하세요.
question = """
For each PowerPoint slide provided above, write a 2-minute script that effectively conveys the key points.
Ensure a smooth flow between slides, maintaining a clear and engaging narrative.
"""

response = my_llm.ask_llm(context, question)

# I do not know 포함일 때 재시도 100번까지 (I do not know 가 들어가면 안되는 경우에만 사용)
count = 0
while "I do not know" in response and count < 100:
    response = my_llm.ask_llm(context, question)
    count += 1
    print("재시도횟수:", count)



재시도횟수: 1
재시도횟수: 2
재시도횟수: 3


In [10]:
print(response)
with open("data/ppt_script.md", "w") as f:
    f.write(response)

Here are the scripts for each PowerPoint slide:

### Slide 1:
"Good morning everyone, today we're going to talk about Machine Learning Pipeline. So, what is it? Well, a machine learning pipeline is the process of building, training, deploying, and maintaining a machine learning model. It's like a workflow that helps us get from raw data to accurate predictions or decisions."

### Slide 2:
"Now, let's discuss what a machine learning pipeline consists of. There are three main types: batch, stream, and realtime. Each has its own use case and requirements."

### Slide 3:
"Batch deployment is ideal for tasks like end-of-day reporting or monthly analytics. It involves processing large volumes of data at scheduled intervals, reducing latency and bandwidth usage."

### Slide 4:
"The infrastructure and integration aspect of a machine learning pipeline is crucial. We need to set up the right environment, integrate the model with existing systems, and ensure seamless communication between compone

In [ ]:
### 엑셀 파일 읽고 분석시키기

# 마이크로소프트 오피스 PPT, Excel, Word 읽기
# pip install unstructured openpyxl python-magic python-pptx
# pip install "unstructured[all-docs]"
# pip install -U nltk

In [11]:
from langchain_community.document_loaders import UnstructuredExcelLoader

In [16]:
loader = UnstructuredExcelLoader("data/sample_ko.xlsx", mode="elements")

docs = loader.load()

print('엑셀 페이지수:', len(docs), '\n\n')
print(f'엑셀 정보:\n{docs}')

엑셀 페이지수: 1 


엑셀 정보:
[Document(metadata={'source': 'data/sample_ko.xlsx', 'file_directory': 'data', 'filename': 'sample_ko.xlsx', 'last_modified': '2025-02-14T07:30:01', 'page_name': 'Data', 'page_number': 1, 'text_as_html': '<table><tr><td>First Name</td><td>Last Name</td><td>City</td><td>Gender</td></tr><tr><td>Brandon</td><td>James</td><td>Miami</td><td>M</td></tr><tr><td>Sean</td><td>Hawkins</td><td>Denver</td><td>M</td></tr><tr><td>Judy</td><td>Day</td><td>Los Angeles</td><td>F</td></tr><tr><td>Ashley</td><td>Ruiz</td><td>San Francisco</td><td>F</td></tr><tr><td>구자</td><td>연</td><td>대한민국</td><td>F</td></tr></table>', 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'category': 'Table', 'element_id': 'f7add31ec38ab6cdc104067295b5b444'}, page_content='First Name Last Name City Gender Brandon James Miami M Sean Hawkins Denver M Judy Day Los Angeles F Ashley Ruiz San Francisco F 구자 연 대한민국 F')]


In [20]:
# 엑셀 정보는 html 테이블 형식으로 구성됨 (랭체인)
context = docs[0].metadata['text_as_html']
print(context)

<table><tr><td>First Name</td><td>Last Name</td><td>City</td><td>Gender</td></tr><tr><td>Brandon</td><td>James</td><td>Miami</td><td>M</td></tr><tr><td>Sean</td><td>Hawkins</td><td>Denver</td><td>M</td></tr><tr><td>Judy</td><td>Day</td><td>Los Angeles</td><td>F</td></tr><tr><td>Ashley</td><td>Ruiz</td><td>San Francisco</td><td>F</td></tr><tr><td>구자</td><td>연</td><td>대한민국</td><td>F</td></tr></table>


In [46]:
question = "Return all entris in the table where Gender is 'F'. Format the response in Markdown. Do not write preambles and explanation."
response = my_llm.ask_llm(context, question)

# I do not know 포함일 때 재시도 100번까지 (I do not know 가 들어가면 안되는 경우에만 사용)
count = 0
while "I do not know" in response and count < 100:
    response = my_llm.ask_llm(context, question)
    count += 1
    print("재시도횟수:", count)

print(response)




| First Name | Last Name | City | Gender |
| --- | --- | --- | --- |
| Judy | Day | Los Angeles | F |
| Ashley | Ruiz | San Francisco | F |
| Guzae | Yoon |대한민국 | F |


In [117]:
question = "Return all entris in the table where Gender is 'M'. Format the response in Markdown."
response = my_llm.ask_llm(context, question)

# I do not know 포함일 때 재시도 100번까지 (I do not know 가 들어가면 안되는 경우에만 사용)
count = 0
while "I do not know" in response and count < 100:
    response = my_llm.ask_llm(context, question)
    count += 1
    print("재시도횟수:", count)

print(response)

| First Name | Last Name | City | Gender |
| --- | --- | --- | --- |
| Brandon | James | Miami | M |
| Sean | Hawkins | Denver | M |

In the table, the entries where 'Gender' is 'M' are:

* Brandon James
* Sean Hawkins


In [121]:
### 워드 파일 읽고 LLM에게 메일쓰게하기

# pip install -U docx2txt
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("data/job_description.docx")

docs = loader.load()

docs

[Document(metadata={'source': 'data/job_description.docx'}, page_content='Job Description - Data Scientist\n\nAt SpiceJet, we rely on data to provide us valuable insights, and to automate our systems and solutions to help us increase revenues, reduce costs and provide improved customer experiences. We are seeking an experienced data scientist to deliver insights and automate our systems and processes. Ideal team member will have mathematical and statistical expertise, experience with modern data science programming languages and machine learning/AI platforms and techniques. You will mine, clean and interpret our data and then develop machine learning models to deliver business value across different parts of the business. \n\nObjectives of this Role\n\nUse Data Science and Machine Learning to increase revenue, reduce costs and increase customer satisfaction.\n\nCollaborate with product design and engineering to develop an understanding of needs\n\nUnderstand where the required data res

In [122]:
context = docs[0].page_content

print(context)

Job Description - Data Scientist

At SpiceJet, we rely on data to provide us valuable insights, and to automate our systems and solutions to help us increase revenues, reduce costs and provide improved customer experiences. We are seeking an experienced data scientist to deliver insights and automate our systems and processes. Ideal team member will have mathematical and statistical expertise, experience with modern data science programming languages and machine learning/AI platforms and techniques. You will mine, clean and interpret our data and then develop machine learning models to deliver business value across different parts of the business. 

Objectives of this Role

Use Data Science and Machine Learning to increase revenue, reduce costs and increase customer satisfaction.

Collaborate with product design and engineering to develop an understanding of needs

Understand where the required data resides and work on ways to extract the relevant data.

Research and devise statistical

In [124]:
# 저는 성종입니다. 저는 최근 IIT를 졸업했으며 자연어 처리 및 머신 러닝에 중점을 두고 있습니다.
# 저는 SpiceJet에서 데이터 과학자 직책에 지원하고 있습니다.
# 구인 게시판이나 출처에 대한 참조를 포함하여 모든 자리 표시자를 제거하고 간결하게 구인 지원 이메일을 작성해 주세요.
question = """
My name is 성종, and I am a recent graduate from IIT with a focus on Natural Language Processing and Machine Learning.
I am applying for a Data Scientist position at SpiceJet.
Please write a concise job application email for me in short, removing any placeholders, including references to job boards or sources.
"""

response = my_llm.ask_llm(context, question)
print(response)

Here is a concise job application email:

Subject: Application for Data Scientist Position at SpiceJet

Dear Hiring Manager,

I am writing to express my interest in the Data Scientist position at SpiceJet, as advertised on [reference source]. With a strong foundation in Natural Language Processing and Machine Learning from my graduate studies at IIT, I am confident that I possess the skills and expertise required to deliver value-added insights and automate systems.

As a detail-oriented and analytical individual with 3+ years of experience in data science, I have developed a solid understanding of predictive modeling techniques, machine learning platforms, and statistical analysis. My proficiency in Python, R, Excel, Tableau, SQL, and programming languages such as design patterns and functional aspects has enabled me to implement effective solutions in real-world scenarios.

In my previous roles, I have demonstrated my ability to collaborate with product design and engineering teams, 

In [125]:
# from 폴더명 import 파일명
from my_scripts import my_translator

# 파일명.함수명()
ko_response = my_translator.toKorean(response)
print(ko_response)

간결한 구직 신청 이메일은 다음과 같습니다.

제목 : SpiceJet의 데이터 과학자 위치 응용 프로그램

친애하는 채용 관리자,

[참조 출처]에 광고 된대로 SpiceJet의 데이터 과학자 위치에 대한 관심을 표명하기 위해 글을 쓰고 있습니다. IIT의 대학원 연구에서 자연어 처리 및 기계 학습의 강력한 토대를 통해 부가가치 통찰력을 제공하고 시스템을 자동화하는 데 필요한 기술과 전문 지식을 보유하고 있다고 확신합니다.

데이터 과학 분야에서 3 년 이상의 경험을 가진 세부 사항 지향적이고 분석적인 개인으로서 예측 모델링 기술, 기계 학습 플랫폼 및 통계 분석에 대한 확실한 이해를 개발했습니다. Python, R, Excel, Tableau, SQL 및 디자인 패턴 및 기능적 측면과 같은 프로그래밍 언어에 대한 숙련도를 통해 실제 시나리오에서 효과적인 솔루션을 구현할 수있었습니다.

이전의 역할에서 저는 제품 설계 및 엔지니어링 팀과 협력하고 이해 관계자의 요구를 이해하며 자동화 된 방식으로 통찰력을 전달할 수있는 능력을 보여주었습니다. 프로젝트 관리에 대한 나의 경험은 또한 작업을 분류하고 의존성을 식별하며 정확한 노력 추정을 제공하는 기술을 갖추게되었습니다.

저는 특히 매출 성장, 비용 절감 및 고객 만족을 위해 데이터 과학을 활용하는 데 SpiceJet의 초점에 중점을 둡니다. 회사의 여러 부분에서 비즈니스 가치를 높이기 위해 기계 학습 및 통계 분석에 대한 전문 지식을 제공 할 수있는 기회에 대해 기쁘게 생각합니다.

제 신청서를 고려해 주셔서 감사합니다. 내 기술과 경험이 Spicejet의 목표와 어떻게 일치하는지 논의하기를 기대합니다.

감사합니다,
 성종
